<a href="https://colab.research.google.com/github/ovapichi/laboratorio-automatico/blob/main/An%C3%A1lisis_de_Celda_de_Carga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requiere de 4 valores de tipo float en el siguiente orden:
1.Cuadriceps Derechos
2.Cuadriceps Izquierdos
3.Isquios Derechos
4.Isquios Izquierdos

In [2]:
!git clone "https://github.com/ovapichi/laboratorio-automatico.git"

Cloning into 'laboratorio-automatico'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 109 (delta 57), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (109/109), 179.39 KiB | 1.55 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%run "/content/laboratorio-automatico/Fpdf.ipynb"

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=2887ee93c0a512393df743c9ac666a383ade731123db3dbdeed29ced7f72fb3b
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [4]:
import fpdf as FPDF
from datetime import datetime
import pandas as pd
import numpy as np
from fpdf import FPDF
from google.colab import drive, files
import shutil
drive.mount('/content/drive')


Mounted at /content/drive


In [107]:
class Celda:
  def __init__(self,cuadriceps_der,cuadriceps_izq,isquios_der,isquios_izq):
    #Constructor
    self.cuadriceps_der=cuadriceps_der
    self.cuadriceps_izq=cuadriceps_izq
    self.isquios_der=isquios_der
    self.isquios_izq=isquios_izq

  def calcular_ratios(self, valor1, valor2):
    #calcula los ratios de cada grupo muscular
    return round(valor1/valor2,2)

  def procesar_simetria(self):
    #Hace una recomendación en función del ratio que obtiene de la función anterior.
    referencia_data=[['Menor a 0.9', 'Debilidad Derecha'],['Entre 0.9 y 1.1', 'Deseable'], ['Mayor a 1.1', 'Debilidad Izquierda']]
    referencia_head=['Referencia', '']
    abs_data=['Extensores Derechos', self.cuadriceps_der], ['Extensores Izquierdos', self.cuadriceps_izq], ['Flexores Derechos', self.isquios_der], ['Flexores Izquierdos', self.isquios_izq]
    abs_head=['Grupo Muscular', 'Valor en kilogramos']
    ratio_cuadriceps=self.calcular_ratios(self.cuadriceps_der,self.cuadriceps_izq)
    ratio_isquios=self.calcular_ratios(self.isquios_der,self.isquios_izq)
    pdf.add_page()
    pdf.set_font("Arial",size=10)
    caratula(jugadora,"Evaluación de la fuerza isométrica de MMII con celda de cargas")
    pdf.add_page()
    pdf.set_y(4)
    pdf.cell(0,1,f'Tabla de valores absolutos',ln=True, align='l')
    pdf.add_table(abs_head,abs_data,5,True)
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0,1,f'Valores de simetría interpierna',ln=True,align='l')
    pdf.set_font('Arial', '', 12)
    pdf.cell(0,1,f'Su valor de simetría de extensores es {ratio_cuadriceps}.',ln=True, align='l')
    pdf.cell(0,1,f'Su valor de simetría de flexores es {ratio_isquios}.',ln=True, align='l')
    pdf.ln()
    pdf.add_table(referencia_head,referencia_data,5,False)
    pdf.ln()

    #print(tabulate(absolutos, headers='firstrow', tablefmt='fancy_grid'))
    #print(f'Su valor de simetría de extensores es {ratio_cuadriceps}. \nSu valor de simetría de flexores es {ratio_isquios}.')
    #print(tabulate(referencia_simetria, headers='firstrow', tablefmt="fancy_grid"))
    return ratio_cuadriceps, ratio_isquios

  def procesar_ratios(self):
    #Toma los ratios anteroposteriores y devuelve la recomendación en función de ellos. Si queda positivo el cuadriceps es mayor.
    ratio_derecho=self.calcular_ratios(self.isquios_der,self.cuadriceps_der)
    ratio_izquierdo=self.calcular_ratios(self.isquios_izq,self.cuadriceps_izq)


    ratio_data=[['Mayor a 0.6', 'Deseable'], ['Menor a 0.6', 'Debilidad de flexores']]
    ratio_head=['Referencia', '']

    pdf.add_page()
    pdf.set_y(4)
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0,1,f'Valores de ratio intrapierna.',ln=True,align='l')
    pdf.set_font('Arial', '', 12)
    pdf.cell(0,1,f'Su H:Q ratio derecho es {ratio_derecho}.',ln=True, align='l')
    pdf.cell(0,1,f'Su H:Q ratio izquierdo es {ratio_izquierdo}.',ln=True, align='l')
    pdf.ln()
    pdf.add_table(ratio_head,ratio_data,5,False)
    pdf.ln()

    return ratio_derecho, ratio_izquierdo

  def ejecutar(self):

    lista=self.procesar_simetria(), self.procesar_ratios()
    lista_nueva=[self.cuadriceps_der, self.cuadriceps_izq, self.isquios_der, self.isquios_izq]
    for i in lista:
      for e in i:
        lista_nueva.append(e)
    print(lista_nueva)

  def ejecutar_historico_celda(self,jugadora,fecha,datos):
    fecha = ["Grupo Muscular"] +fecha
    hist_head=fecha


    # Crear una lista de listas donde cada lista representa los valores de una posición

    data=['Extensores D']+datos[0],['Extensores I']+datos[1],['Flexores D']+datos[2],['Flexores I']+datos[3],['Simetría D']+datos[4],['Simetría I']+datos[5],['H:Q Ratio D']+datos[6],['H:Q Ratio I']+datos[7]
    hist_data=[data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7]]



    pdf.add_page()
    pdf.set_font("Arial",size=10)
    caratula(jugadora,"Resúmen histórico celda de cargas")
    pdf.add_page()
    pdf.set_y(4)
    pdf.set_font('Arial', 'B', 14)

    pdf.cell(0,1,f'Tabla de valores',ln=True, align='l')
    pdf.ln()
    pdf.add_table(hist_head,hist_data,4,True)


In [108]:
# pdf = PDF("p","cm","A4")

 #anita=Celda(50, 55, 30, 35)
 #data=[[20.5, 30.62], [23.9, 31.52], [70.1, 63.15], [66.6, 58.35], [1.05, 1.08], [0.86, 0.97], [0.29, 0.48], [0.36, 0.54]]

 #fecha=['2022-09', '2023-03', '2023-11']

 #anita.ejecutar_historico_celda('Agustina Fridman', fecha, data)


In [109]:
#exportar("hola","/content/sample_data")